<a href="https://colab.research.google.com/github/frankausberlin/notebook-collection/blob/main/snippetpearls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table width=1000><tr></tr><tr><td align='center'><font size='+10' color='#005F6A'><b>
SnippetPearls</b></font></td></tr><tr></tr><tr><td align='center'><p>
This is a jupyter notebook with several and I hope useful<br> code snippets around very interesting technologies.</td></tr><tr></tr><tr><td align='center'>
optimized for google colab</td></tr></table><br><br>

---

<br>
<font size='+2'>Installation<br>

It's realy easy:
1. Copy the url of your (or this) notebook,
2. go to menu tools / settings / websites
3. and past your url to field 'Custom snippet notebook url'.

It will now, after reloading the page, treat all text cells with chapter entries ('#') and their subsequent code cells as a snippet and display them in the colab snippet selector


# _sp: chat gpt

In [ ]:
# *******************************************************************************************************************************************************************************************
# *** Colab Forms ***************************************************************************************************************************************************************************
# *******************************************************************************************************************************************************************************************

#@markdown <font size='+2' color='#005F6A'>**chatGPT**</font> <font size='-1'>from <b>openai</b></font>
#@markdown <br>A little **gui** for chat gpt with optional **yaml** persistence on **gdrive** or **local runtime** or both **synchronized**
#@markdown * This snippet needs a **personal api key** (<a href='https://openai.com/pricing'>pricing</a>) - you can get it <a href='https://platform.openai.com/account/api-keys'>here</a> and you can control your usage <a href='https://platform.openai.com/account/usage'>here</a>.
#@markdown * You can manage limitations <a href='https://platform.openai.com/account/billing/limits'>here</a>. There are <a href='https://platform.openai.com/docs/models'>many models</a> to choose from and a big <a href='https://community.deeplearning.ai/'>community</a>.
#@markdown * If you want dive deeper into **prompt engineering** there is a litte <a href='https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/'>course from deeplearning.ai</a>.
#@markdown * A **```gptchat``` folder** is created/used to store yamls and **synchronize** if local.
#@markdown * On local runtime you have an **optional rclone gdrive** support, to configure look <a href="https://rclone.org/">here</a>.<br>If you put a <b>local gdrive folder</b> at start it <b>automaticly sync gdrive</b> to local and do a sync to gdrive by each 'send'.
#@markdown * On **hosted runtime** you have an **optional gdrive** support too - uses the colab gdrive integration.
#@markdown * You can use **categories** to **organize** your dialogs - every categorie has a **subfolder** with its own yamls.
#@markdown ---
#@markdown <font color='black' size='+1'><b>Local runtime</b></font><br><font color='#005F6A'>In **hosted runtime** this parameter are **ignored**.</font>
local_cloud_folder = '~/labor/gdrive'   #@param {type:"string"}
rclone_name  = 'gdrive:' # fix use
#@markdown <font color='black' size='+1'><b>Chat</b></font><br><font color='#005F6A'>If **categorie_name** is not empty, it is used as a **subfolder** in the gptchat folder and its content is displayed in the **recent chats** selector.</font>
categorie_name  = 'docker'   #@param {type:"string"}



# *******************************************************************************************************************************************************************************************
# *** Imports *******************************************************************************************************************************************************************************
# *******************************************************************************************************************************************************************************************
import                    os, subprocess, getpass, yaml, time
from ipywidgets   import  HTML, VBox, Box, Tab, Textarea, Button, Layout, Dropdown, HBox, Text, Checkbox, Label, Accordion
from datetime     import  datetime
from pathlib      import  Path



# *******************************************************************************************************************************************************************************************
# *** Prompts *******************************************************************************************************************************************************************************
# *******************************************************************************************************************************************************************************************
prompt_system = "You are a friendly assistant, you receive questions or instructions in any language and respond in English.\n"+\
                "You try to break down your instructions into steps, work through them and check the results of each step before presenting the overall result."
prompt_title  = "You are an assistant and you are creating a title for the following chat history.\n"+\
                "The title should list all topics of the chat history well.\n"+\
                "Your output must have less than 9 words and is only the title and everything in English."



# *******************************************************************************************************************************************************************************************
# *** snippet ButtonBox *********************************************************************************************************************************************************************
# *******************************************************************************************************************************************************************************************
class ButtonBox ():
  """ An alternative for the Tab widget
  """
  def _clicker (self, b):
    # search clicked button and remember
    for i, description in enumerate (self.descriptions):
      if b.tooltip == description:
        self.selected = i
        self.button   = b

    # unselect (color) all buttons and select new (list of colors here: https://www.quackit.com/css/css_color_codes.cfm)
    for b in self.buttons: b.style={'button_color':None}
    self.buttons[self.selected].style={'button_color':self.color}

    # fire event
    if self.clicker: self.clicker (self)

  def __init__ (self, descriptions, clicker=None, maxchar=80, color='powderblue'):
    # remember stuff
    self.clicker, self.maxchar, self.color, self.selected, self.button = clicker, maxchar, color, None, None

    # build buttons
    self.buildButtons (descriptions)

  def buildButtons (self, descriptions):
    # list to sorted set
    self.descriptions = sorted (descriptions)

    # make buttons
    self.buttons = [Button (description = i if len (i) <= self.maxchar else f'{i[:(self.maxchar-3)]}...',
                            layout      = Layout(width='auto', height='21px'),
                            tooltip     = f'{i}')
                    for i in self.descriptions]

    # put them in a box
    self.box = Box (layout = Layout (display='flex', flex_flow='wrap'), children = self.buttons)

    # bind event
    for button in self.buttons: button.on_click (self._clicker)

  def removeSelected (self):
    if self.selected != None:
      self.box.children = [*self.box.children[:self.selected], *self.box.children[self.selected+1:]]
      self.button = self.selected = None




# *******************************************************************************************************************************************************************************************
# *** class *********************************************************************************************************************************************************************************
# *******************************************************************************************************************************************************************************************
class ChatGPT:
  """ A little chat client with yaml persistence
  widget-structure:
  hm_state              - HTML      - link api-key / done message
  ac_main               - Accordion - main accordion with 'new chat' and 'recent chats'
    + child0            - VBox      - recent chats
      + child0          - Box       - chat selector (managed by ButtonBox)
      + child1          - HTML      - cosmetic <hr>
      + hb_tools        - HBox      - Toolbar with buttons and html
      |   + child0      - HTML      - yaml filename
      |   + child1      - Box       - cosmetic (for html left, buttons right)
      |   + bu_conChat  - Button    - continue chat
      |   + bu_delChat  - Button    - delete chat
      + child3          - TextArea  - display chat
    + child1 VBox       - VBox      - new chat
      + hb_config       - HBox      - the config line
      |   + child0      - Dropdown  - the models
      |   + child1      - Text      - temperature
      |   + child2      - Checkbox  - write to yaml
      |   + child3      - Checkbox  - use cloud / local runtime
      |   + child4      - HTML      - filename and location yaml
  |   + ta_system       - TextArea  - system instruction
      + ta_titprm       - TextArea  - title prompt
      + child3          - HTML      - cosmetic <hr>
      + vb_main         - VBox      - growing Textareas
      |   + ta_user     - TextArea  - first question
      |   *+ child      - TextArea  - assistant (response)    # current last two textareas in list are
      |    + child      - TextArea  - user (next question)    # accessable in self.ta_assist and self.ta_user
      + hb_control      - HBox      - send button and waiter
          + bu_send     - Button    - send
          + hm_waiter   - HTML      - wait animation
  """
# *******************************************************************************************************************************************************************************************
# *** init / helper *************************************************************************************************************************************************************************
# *******************************************************************************************************************************************************************************************
  def __init__(self, local_cloud_folder=None, rclone_name=None, categorie_name=None):
    self.hm_state   = HTML      ("""<font size='+1' color='purple'><a href='https://platform.openai.com/account/api-keys'>api key here</a> - copy and paste your key in the input field.</font>""")
    self.bu_send    = Button    (description='>>', layout=Layout(width='60px'), style={'button_color':'lightgreen'}, tooltip='Send user prompt to gpt, show answer and write yaml if selected.')
    self.ta_system  = Textarea  (description='system:', value=prompt_system, layout=Layout(width='auto',height='80px'))
    self.ta_titprm  = Textarea  (description='title prompt:', value=prompt_title, layout=Layout(width='auto',height='80px'))
    self.ta_user    = Textarea  (description='user:', value='tell a joke', layout=Layout(width='auto',height='300px'))
    self.vb_main    = VBox      (children=[self.ta_user], layout=Layout(width='1200px'))
    self.hb_config  = HBox      ([Dropdown (description='model:'),
                                  Text     (description='temperature', value='1.0', layout=Layout(width='130px')),
                                  Checkbox (description='use gdrive', layout=Layout(width='220px')),
                                  Checkbox (description='write yaml', layout=Layout(width='180px')),
                                  HTML     (value='') ])
    self.hm_waiter  = HTML      ('',layout=Layout(width='200px'))
    self.hb_control = HBox      ([self.bu_send,self.hm_waiter],layout=Layout(width='auto'))
    self.bu_conChat = Button    (description='continue chat', disabled=True,style={'button_color':'khaki'})
    self.bu_delChat = Button    (description='delete chat', disabled=True,style={'button_color':'khaki'})
    self.hb_tools   = HBox      ([HTML(), Box(layout=Layout(flex='1 1 auto')), self.bu_conChat, self.bu_delChat])
    self.ac_main    = Accordion (children=[VBox(), VBox ([self.hb_config, self.ta_system, self.ta_titprm, HTML('<hr>'), self.vb_main, self.hb_control])])
    self.ac_main      .set_title  (0,f'recent chats - {categorie_name}' if categorie_name and len (categorie_name) else 'recent chats')
    self.ac_main      .set_title  (1,'new chat')
    self.ac_main      .selected_index = None

    # init / display state-html
    self.messages, self.yamlFile, self.chats, self.local_cloud_folder, self.rclone_name = [], '', {}, local_cloud_folder, rclone_name
    self.categorie_name = '/'+categorie_name if categorie_name and len (categorie_name) else ''
    display (self.hm_state)

    # runtime
    self.initializeRuntime ()

    # display chat home
    self.hb_config.children[-1].value = f'&nbsp;&nbsp;&nbsp; in &nbsp; {self.chatHome}'

    # local runtime
    if self.isLocal:
      self.hb_config.children[2].description = 'local runtime'
      self.hb_config.children[2].value       = True
      self.hb_config.children[2].disabled    = True

    # load yaml / create chat selector / connect openai
    try:    self.chats = yaml.safe_load (open(f'{self.chatHome}/chats.yaml', 'r', encoding='unicode_escape'))
    except: self.chats = {}
    self.createChatSelector()
    self.do_openai()

    # bind events
    if not self.hb_config.children[2].disabled:
      self.hb_config.children[2] .observe   (self.useGdrive,   names=['value'])
    self.hb_config.children[3]   .observe   (self.writeToYaml, names=['value'])
    self.bu_send                 .on_click  (self.send_click)
    self.bu_conChat              .on_click  (self.continue_click)
    self.bu_delChat              .on_click  (self.delete_click)

    # display main accordion
    display (self.ac_main)


  def initializeRuntime (self):
    # solve ~ in cloud folder / check: local runtrime / rclone installed / valid remote name / valid local cloud folder
    if '~' in self.local_cloud_folder: self.local_cloud_folder = self.local_cloud_folder.replace ('~', os.path.expanduser('~'))
    self.isLocal   = (os.path.expanduser('~') != '/root')
    self.doRclone  = (os.system ('rclone -V >/dev/null 2>&1')==0 and self.rclone_name!='' and self.local_cloud_folder!='' and self.isLocal)
    if self.doRclone and not self.rclone_name in str(subprocess.check_output("rclone listremotes", shell=True)):
      raise Exception (f'\x1b[91m{self.rclone_name} is no valid remote name')
    if self.doRclone and not Path(self.local_cloud_folder).exists():
      raise Exception (f'\x1b[91mfolder {self.local_cloud_folder} does not exist')

    # chat home folder
    if self.isLocal:  self.chatHome = '.' if not self.doRclone else self.local_cloud_folder
    else:             self.chatHome = '/content'
    if not 'gptchat' in os.listdir (f'{self.chatHome}'):  os.mkdir (f'{self.chatHome}/gptchat')
    if not 'gptchat' in self.chatHome:                    self.chatHome += f'/gptchat{self.categorie_name}'

    # sync
    if self.isLocal and self.doRclone and 'gptchat' in str (subprocess.check_output(f"rclone lsd  {self.rclone_name}", shell=True)):
      try: os.system(f"rclone sync {self.rclone_name}/gptchat {self.local_cloud_folder}/gptchat")
      except: pass


  def createChatSelector (self):
    # create chat selector with recent chats and textarea to display
    self.bb_selrecent = ButtonBox (set([self.chats[fname]['name'] for fname in self.chats]), clicker=self.chat_click)
    self.ac_main.children[0].children = [self.bb_selrecent.box, HTML(value='<hr>'), self.hb_tools,
                                         Textarea(layout=Layout(width='auto',height='600px'))]


# *******************************************************************************************************************************************************************************************
# *** events buttons +++++++++++++***********************************************************************************************************************************************************
# *******************************************************************************************************************************************************************************************
  def delete_click (self, b):
    # remove button / buttons disable / clear textarea
    self.bb_selrecent.removeSelected ()
    self.bu_conChat.disabled = self.bu_delChat.disabled = True
    self.ac_main.children[0].children[3].value = self.hb_tools.children[0].value = ''

    # delete file & intern yamls / rewrite chats.yaml / sync removing
    os.system (f'rm {self.chatHome}/{self.fname}')
    del self.chats[self.fname]
    yaml.dump (self.chats, open (f'{self.chatHome}/chats.yaml',  'w'))
    if self.rclone_name and os.path.expanduser('~') != '/root':
      try:    os.system(f"rclone sync {self.local_cloud_folder}/gptchat {self.rclone_name}/gptchat")
      except: pass

    # special: continue clicked before delete
    if self.yamlFile == self.fname:
      self.ac_main.set_title(1,'new chat')
      self.hb_config.children[3].value, self.hb_config.children[3].disabled = False, False
      self.hb_config.children[3].observe (self.writeToYaml, names=['value'])
      self.yamlFile          = ''
      self.ta_user           = Textarea (description='user:', value='', layout=Layout(width='auto',height='300px'))
      self.vb_main.children  = [self.ta_user]


  def continue_click (self, b):
    # new title / checkbox writeyaml
    self.ac_main.set_title(1,self.bb_selrecent.button.tooltip)
    self.hb_config.children[3].unobserve_all()
    self.hb_config.children[3].value, self.hb_config.children[3].disabled = True, True

    # set new yaml
    self.yamlFile = f'{self.fname}'
    try:     self.messages = yaml.safe_load (open(f'{self.chatHome}/{self.fname}', 'r'))
    except:  self.messages = {}

    # rebuild textareas
    self.ta_system.value = self.messages[0]['content']
    taList = []
    for i in range (len(self.messages)-2):
      taList.append ( Textarea (description=self.messages[i+1]['role'], value=self.messages[i+1]['content'], layout=Layout(width='auto',height='65px')) )
    self.ta_assist  = Textarea (description='assistant:', value=self.messages[-1]['content'], layout=Layout(width='auto',height='300px'))
    self.ta_user    = Textarea (description='user:', value='', layout=Layout(width='auto',height='300px'))
    self.vb_main.children = [*taList, self.ta_assist, self.ta_user]

    # select chat accordion tab
    self.ac_main.selected_index = 1


  # send button click
  def send_click (self, b):
    # waiter
    self.hm_waiter.value, b.disabled = """<marquee style='width: 100%; color: blue;'>waiting for response...</marquee>""", True

    # init messages
    if len (self.messages) == 0:  self.messages = [{'role':'system', 'content':f'{self.ta_system.value}'}]

    # add new question and its response to messages
    self.messages.append ({'role':'user', 'content':self.vb_main.children[-1].value})
    response = self.get_completion_from_messages (self.messages,
                                                  model       = self.hb_config.children[0].value,
                                                  temperature = float(self.hb_config.children[1].value)
                                                 ).choices[0].message.content
    self.messages.append ({'role':'assistant', 'content':f'{response}'})

    # make old textareas small and create new ones
    for ta in self.vb_main.children: ta.layout = Layout(width='auto',height='65px')
    self.ta_assist         = Textarea (description='assistant:', value=response, layout=Layout(width='auto',height='300px'))
    self.ta_user           = Textarea (description='user:', value='', layout=Layout(width='auto',height='300px'))
    self.vb_main.children  = [*self.vb_main.children,self.ta_assist,self.ta_user]

    # reset waiter / ask title / write yaml
    self.hm_waiter.value, b.disabled = '', False

    tmpSys   = {'role':'system', 'content':self.ta_titprm.value}
    response = self.get_completion_from_messages ([tmpSys, *self.messages[1:]],
                                                  model       = "gpt-3.5-turbo", # self.hb_config.children[0].value,
                                                  temperature = float(self.hb_config.children[1].value)
                                                 ).choices[0].message.content
    while response[0]  in """ !§$%&()=?`'{}[]@~+*<>|\/-:"'#.,;_""": response = response[1:]
    while response[-1] in """ !§$%&()=?`'{}[]@~+*<>|\/-:"'#.,;_""": response = response[:-1]
    self.ac_main.set_title  (1,f'{response}')
    self.writeToYaml (None)


  def chat_click (self, bb):
    # create content for textarea
    fname    = [fn for fn in self.chats if self.chats[fn]['name'] == self.bb_selrecent.button.tooltip][0]
    recent   = yaml.safe_load (open(f'{self.chatHome}/{fname}', 'r'))
    con      = f"{self.chats[fname]['name']}\n\n"
    con     += f"File:       {fname}\n"
    con     += f"Model:      {self.chats[fname]['model']}\n"
    con     += f"Temperatur: {self.chats[fname]['temp']}\n\n"
    con     += f"System: {recent[0]['content']}\n\n"
    con     += f"\n{'*'*300}\n\n"
    for i in recent[1:]:
      con   += f">>>{i['role']}>>>\n"
      con   += f"{i['content']}\n"
      con   += "\n" if i['role'] != 'assistant' else f"\n{'*'*300}\n\n"
    self.ac_main.children[0].children[3].value = con

    # enable buttons / remember + show filename
    self.bu_conChat.disabled = self.bu_delChat.disabled = False
    self.fname = fname
    self.hb_tools.children[0].value = f"<font size='+2' color='Blue'>{fname}</font>"



# *******************************************************************************************************************************************************************************************
# *** yaml / gdrive / openai ****************************************************************************************************************************************************************
# *******************************************************************************************************************************************************************************************
  # checkbox write yaml observe
  def writeToYaml (self,cb):

    # call by checkbox
    if cb:
      # yaml file name / disable cb
      tmp_dat = datetime.now().strftime("%Y%m%d-%H%M%S")
      self.yamlFile                       = f'gpt_chat_{tmp_dat}.yaml'
      self.hb_config.children[4].value    = f'{self.yamlFile} &nbsp;&nbsp;&nbsp; in &nbsp; {self.chatHome}'
      self.hb_config.children[3].disabled = True
      self.hb_config.children[2].disabled = True

    # write yaml
    if self.messages and self.yamlFile:
      if self.categorie_name and not Path(self.chatHome).exists():
        os.mkdir (self.chatHome)

      self.chats[self.yamlFile] = {'name'  :f"{self.yamlFile[11:17]} - {self.ac_main.get_title(1)}",
                                   'model' :f'{self.hb_config.children[0].value}',
                                   'temp'  :f'{self.hb_config.children[1].value}'}
      yaml.dump (self.messages, open (f'{self.chatHome}/{self.yamlFile}', 'w'))
      yaml.dump (self.chats,    open (f'{self.chatHome}/chats.yaml',  'w'))

      # sync local to cloud
      if self.doRclone:
        try: os.system(f"rclone sync {self.local_cloud_folder}/gptchat {self.rclone_name}/gptchat")
        except: pass


  # checkbox 'use gdrive' observe
  def useGdrive (self,b):
    # colab gdrive stuff
    try:
      from google.colab import drive
      if not 'gdrive'  in os.listdir('/'):  drive.mount ('/gdrive')
      self.chatHome = f'/gdrive/MyDrive'
    except:
      self.hb_config.children[3].value       = False
      self.hb_config.children[3].description = 'failed'
    if not 'gptchat' in os.listdir(self.chatHome): os.mkdir (f'{self.chatHome}/gptchat')
    if not 'gptchat' in self.chatHome: self.chatHome += f'/gptchat{self.categorie_name}'
    self.hb_config.children[-1].value    = f'{self.yamlFile} &nbsp;&nbsp;&nbsp; in &nbsp; {self.chatHome}'
    self.hb_config.children[2].disabled  = True

    # load chats from gdrive
    try:    self.chats = yaml.safe_load (open(f'{self.chatHome}/chats.yaml', 'r', encoding='unicode_escape'))
    except: self.chats = {}
    self.createChatSelector()

  # init openai connection
  def do_openai (self):
    # openai api key input / widget models (+default) / done state
    os.environ["OPENAI_API_KEY"]        = openai.api_key = getpass.getpass (prompt='Your API key: ') if not openai.api_key else openai.api_key
    self.hb_config.children[0].options  = [m.id for m in openai.models.list() if 'gpt' in m.id]
    self.hb_config.children[0].value    = 'gpt-4'
    self.chatclient                     = OpenAI()
    self.hm_state.value                 = """<font size='+2' color='powderblue'>Set API-Key done!</font>"""

  # openai endpoint
  def get_completion_from_messages (self, messages, model="gpt-3.5-turbo", temperature=0):
    """ see https://platform.openai.com/docs/api-reference/introduction """
    return self.chatclient.chat.completions.create ( model       = model,
                                                     messages    = messages,
                                                     temperature = temperature )
try:              import  openai
except:                   os.system ('pip install openai'); import openai
from     openai   import  OpenAI



# *******************************************************************************************************************************************************************************************
chat = ChatGPT (local_cloud_folder=local_cloud_folder, rclone_name=rclone_name, categorie_name=categorie_name)




# _sp: command shell

In [ ]:
#@markdown <font size='+2' color='#005F6A'>**command shell**</font>
#@markdown * using [jQuery Terminal](https://terminal.jcubic.pl/) - example on [github](https://github.com/alessandrobessi/colab-shell)
#
%matplotlib inline
import                            os
from   google.colab       import  output                # for register callback
from   IPython.display    import  display, HTML, JSON   # draw the shell box with IPython
from   subprocess         import  getoutput             # to exectute command and get output
from   ipywidgets         import  Accordion, Textarea   # used for history

width  = 1600 #@param {type: "slider", min: 400, max: 2000, step: 100}
height = 400 #@param {type: "slider", min: 100, max: 1000, step: 100}
printHistory = True #@param {type:"boolean"}


# function for invoke-callback-mechanism
def shell(command):
  global taHistory, acHistory
  # put command to history textarea and set accordion title
  if printHistory:
    taHistory.value += command + '\n'
    acHistory.set_title (0,f'history - last command: {command.split(" ")[0]}')

  # catch cd-command
  if command.startswith('cd'):
    path = command.strip().split(maxsplit=1)[1]
    os.chdir(path)
    return JSON([''])

  # all other commands
  return JSON([getoutput(command)])

# register to invoke
output.register_callback('shell', shell)

# create command tab using javascript with 'display(HTML(...'
display(HTML('''
  <div id=term_demo></div>
  <script src="https://code.jquery.com/jquery-latest.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/jquery.terminal/js/jquery.terminal.min.js"></script>
  <link href="https://cdn.jsdelivr.net/npm/jquery.terminal/css/jquery.terminal.min.css" rel="stylesheet"/>
  <script>
    $('#term_demo').terminal(async function(command) {
        if (command !== '') {
            try {
                let res = await google.colab.kernel.invokeFunction('shell', [command])
                let out = res.data['application/json'][0]
                this.echo(new String(out))
            } catch(e) {
                this.error(new String(e));
            }
        } else {
            this.echo('');
        }
    }, {
        greetings: 'Welcome to Colab Shell',
        name: 'colab_shell',
        height: '''+str(height)+''',
        width: '''+str(width)+''',
        prompt: 'colab > '
    });
  </script>'''))

# History
taHistory = Textarea (layout={'width':'auto', 'height':'200px'})
acHistory = Accordion (children=[taHistory])
acHistory.set_title (0,'history - click to expand')
acHistory.selected_index = None
if printHistory:
  display (acHistory)

# _sp: image search

In [ ]:
#@markdown <font size='+2' color='#005F6A'>**image search**</font><br>
#@markdown This little snippet does a picture search with DuckDuckGo (jpg, png, gif, bmp) and show the results (thumbnails).
#@markdown * It creates a folder content/images (local runtime: ~/labor/content/images) and there a new folder for every search.
#@markdown * The image thumbnails (and optional the images) will be downloaded in the search folder.
#@markdown * It will generate a bash script (easy remove bad pictures) for downloading the images later (or again).

import                            time, json, re, ipywidgets, os, \
                                  requests, matplotlib
from matplotlib           import  pyplot as plt
from urllib.parse         import  urlencode, quote, urlparse, urlunparse
from urllib.request       import  build_opener, Request
from urllib.error         import  HTTPError, URLError
from google.colab         import  widgets
from PIL                  import  Image
from tqdm                 import  tqdm

query                     = 'angry cat' #@param {type:"string"}
results                   = 10 #@param {type: "slider", min: 1, max: 999, step: 1}
downloadImages            = False #@param {type:"boolean"}
                            #@markdown **thumbnails display**
columns                   = 10 #@param {type: "slider", min: 1, max: 20, step: 1}
rows                      = 1 #@param {type: "slider", min: 1, max: 100, step: 1}

##########################################################################################################
########################## reversed from https://course.fast.ai/ #########################################
##########################################################################################################
url_default_headers = {
    "Accept":
    "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Accept-Language": "en-US,en;q=0.9",
    "Cache-Control": "max-age=0",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"
}

ExceptionsHTTP = {}

def urlquote(url):
    "Update url's path with `urllib.parse.quote`"
    subdelims = "!$&'()*+,;="
    gendelims = ":?#[]@"
    safe = subdelims+gendelims+"%/"
    p = list(urlparse(url))
    p[2] = quote(p[2], safe=safe)
    for i in range(3,6): p[i] = quote(p[i], safe=safe)
    return urlunparse(p)

def loads(s, **kw):
    "Same as `json.loads`, but handles `None`"
    if not s: return {}
    try: import ujson as json
    except ModuleNotFoundError: import json
    return json.loads(s, **kw)

def urlwrap(url, data=None, headers=None):
    "Wrap `url` in a urllib `Request` with `urlquote`"
    return url if isinstance(url,Request) else Request(urlquote(url), data=data, headers=headers or {})

def urlopener():
    _opener = build_opener()
    _opener.addheaders = list(url_default_headers.items())
    return _opener

def urlopen(url, data=None, headers=None, timeout=None, **kwargs):
    "Like `urllib.request.urlopen`, but first `urlwrap` the `url`, and encode `data`"
    if kwargs and not data: data=kwargs
    if data is not None:
        if not isinstance(data, (str,bytes)): data = urlencode(data)
        if not isinstance(data, bytes): data = data.encode('ascii')
    try: return urlopener().open(urlwrap(url, data=data, headers=headers), timeout=timeout)
    except HTTPError as e:
        e.msg += f"\n====Error Body====\n{e.read().decode(errors='ignore')}"
        raise

def urlread(url, data=None, headers=None, decode=True, return_json=False, return_headers=False, timeout=None, **kwargs):
    "Retrieve `url`, using `data` dict or `kwargs` to `POST` if present"
    try:
        with urlopen(url, data=data, headers=headers, timeout=timeout, **kwargs) as u: res,hdrs = u.read(),u.headers
    except HTTPError as e:
        if 400 <= e.code < 500: raise ExceptionsHTTP[e.code](e.url, e.hdrs, e.fp) from None
        else: raise

    if decode: res = res.decode()
    if return_json: res = loads(res)
    return (res,dict(hdrs)) if return_headers else res

def isddg(term, max_images=200):
    "Search for `term` with DuckDuckGo and return a unique urls of about `max_images` images"
    assert max_images<1000
    url = 'https://duckduckgo.com/'
    res = urlread(url,data={'q':term})
    searchObj = re.search(r'vqd=([\d-]+)\&', res)
    assert searchObj
    requestUrl = url + 'i.js'
    params = dict(l='us-en', o='json', q=term, vqd=searchObj.group(1), f=',,,', p='1', v7exp='a')
    xu,data = list(),{'next':1} # little change: don't use the mysterious L class
    headers = dict(referer='https://duckduckgo.com/')
    while len(xu)<max_images and 'next' in data:
        try:
            res = urlread(requestUrl, data=params, headers=headers)
            data = json.loads(res) if res else {}
            for i in range(len(data['results'])):
              xu.append(data['results'][i])
            requestUrl = url + data['next']
        except (URLError,HTTPError): pass
        time.sleep(1)
    return xu[:max_images]
##########################################################################################################
##########################################################################################################
##########################################################################################################

# download the ddg results page (links)
links                 = isddg (query,results)
if downloadImages:    print (f'checking {len(links)} links - start download thumbnails and images')
else:                 print (f'checking {len(links)} links - start download thumbnails')

# mkdir
qfolder       = query.replace (' ','')
if os.path.exists('/content'):
  os.makedirs   ('/content/images',exist_ok=True)
  os.makedirs   (f'/content/images/{qfolder}',exist_ok=True)
  os.chdir      (f'/content/images/{qfolder}')
else:
  os.makedirs   (f'{os.path.expanduser("~")}/labor/content',exist_ok=True)
  os.makedirs   (f'{os.path.expanduser("~")}/labor/content/images',exist_ok=True)
  os.makedirs   (f'{os.path.expanduser("~")}/labor/content/images/{qfolder}',exist_ok=True)
  os.chdir      (f'{os.path.expanduser("~")}/labor/content/images/{qfolder}')


# inits
pos, bad, thumbs, bash = 0, 0, [], f'#!/bin/bash\nmkdir -p {qfolder}\n'

# loop the links
for all in tqdm(links):

  # extract infos
  url  = all['image']
  turl = all['thumbnail']
  ext  = url.split('.')[-1]
  if '?' in ext: ext = ext.split('?')[0]
  dest = f"{qfolder}{pos-bad}.{ext}"
  pos+=1

  # check extension
  if ext.lower() in 'jpg png gif bmp':

    # download image und thumbnail
    try:

      # image
      if downloadImages:
        im = Image.open(requests.get(url, stream = True).raw)
        im.save(dest)

      # bash script entry
      bash += f'# Picture {dest}\ncurl "{url}" --output {qfolder}/{dest}\n'

      # thumbnail
      im = Image.open(requests.get(turl, stream = True).raw)
      im.save('thumb'+dest)
      thumbs.append('thumb'+dest)

    # count bad links
    except Exception:
      bad += 1
  else:
    bad += 1

# output infos
if not downloadImages:  txt = f'downloaded {len(thumbs)} thumbnails'
else:                   txt = f'downloaded {len(thumbs)} images and thumbnails'
print                   (f'\n\n\x1b[32m{txt}\x1b[0m - \x1b[91mfound {bad} bad links!!!\x1b[0m')

# write bash
if os.path.exists('/content'):
  print    (f'Create bash script: \x1b[34m/content/images/dl_{qfolder}.sh\x1b[0m')
  open     (f'/content/images/dl_{qfolder}.sh', 'w').write(bash)
else:
  print    (f'Create bash script: \x1b[34m~/labor/content/images/dl_{qfolder}.sh\x1b[0m')
  open     (f'{os.path.expanduser("~")}/labor/content/images/dl_{qfolder}.sh', 'w').write(bash)


# build the thumbnails tabs
titles, len_thumbs = [], len(thumbs)
for i in range(0,len_thumbs,(columns*rows)):
  x = ((i//(columns*rows))+1)*(columns*rows)-1
  if x >= len_thumbs: x = len_thumbs-1
  titles.append(f'{i}-{x}')
tabs = widgets.TabBar(titles)

# title font
matplotlib.rc('font', **{'sans-serif':    'DejaVu Sans',
                          'family':        'sans-serif',
                          'weight':        'bold',
                          'size':          (columns+10 if columns > 12 else 16) })

# fill the tabs
for i,t in enumerate(titles):
  with tabs.output_to (t):

    # figure dimension (inch)
    fig = plt.figure(figsize=(4*columns, 4.25*rows))

    # all images for the tab i
    for j in range(columns*rows):
      x = i*(columns*rows)+j

      # subplot image
      if x < len_thumbs:
        fig.add_subplot(rows, columns, j+1, xticks=[], yticks=[], title=f'{x}')
        im = Image.open (thumbs[x])
        plt.imshow(im)

# cd to image folder
if os.path.exists('/content'): os.chdir (f'/content/images')
else: os.chdir (f'{os.path.expanduser("~")}/labor/content/images')

# _sp: dictor

In [ ]:
#@markdown <font size='+2' color='#005F6A'>**dictor**</font><br>
#@markdown * A little dict editor using ipywidgets - easy expandable and customizable.
#@markdown ```python
#@markdown sample = {'a':'one','b':'two'}
#@markdown doit   = lambda w, d: print (f'doit {type(w)} {d}')
#@markdown dictor(sample,doit).show() # or simple: dictor().show()
#@markdown ```

from ipywidgets import Tab, Text, Box, VBox, HBox, Button, Textarea, Layout, Label

class dictor ():
  def doitClick (self, b):
    try:    self.doit (self.vboxMain, self.data)
    except: pass
  def deleteItem (self, b):
    try:
      self.data.pop (self.textKey.value);
      self.buildTab ()
    except: pass
  def storeItem (self, b):
    try:
      if self.textKey.value:
        self.data[self.textKey.value] = self.textareaValue.value
      self.buildTab ()
    except: pass
  def keyChange (self, change):
    titles                            = [self.tabElements.get_title (i+1) for i in range(len(self.tabElements.children)-1)]
    pos                               = titles.index(self.textKey.value) + 1 if self.textKey.value in titles else 0
    self.tabElements                  .unobserve_all ()
    self.tabElements.selected_index   = pos
    self.textareaValue.value          = str(self.data[titles[pos-1]]) if pos else ''
    self.tabElements                  .observe (self.tabSelect, names='selected_index')
  def tabSelect (self, change):
    self.textKey.value                = self.tabElements.get_title(self.tabElements.selected_index) if self.tabElements.selected_index else ''
    self.textareaValue.value          = str (self.data[self.textKey.value]) if self.textKey.value in self.data.keys() else ''
  def show (self):
    display                           (self.vboxMain)
  def buildTab (self):
    self.tabElements                  .unobserve_all ()
    self.tabElements.children         = self.tabElements.children[:1]
    if len (self.data):
      for i, (k, v) in                enumerate (self.data.items()):
        self.tabElements.children     = [*self.tabElements.children, Label (value=str (v))]
        self.tabElements              .set_title (i+1, k)
    self.tabElements.selected_index   = 0
    self.textareaValue.value          = ''
    self.textKey.value                = ''
    self.tabElements                  .observe (self.tabSelect, names='selected_index')
  def __init__(self, dic=None, doit=None):
    self.doit,self.data = doit,dic    if dic else {}
    self.buttonStore    = Button      (description='Store')
    self.buttonDelete   = Button      (description='Delete')
    self.buttonDoit     = Button      (description='Do it')
    self.textKey        = Text        (description='Key',    layout=Layout(width='550px'))
    self.textareaValue  = Textarea    (description='Value',  layout=Layout(width='550px',height='100px'))
    self.boxButtons     = Box         (layout=Layout(display='flex', flex_flow='wrap'))
    self.tabElements    = Tab         (children=[Label()],layout=Layout(display='flex', flex_flow='wrap') )
    self.vboxControls   = VBox        (children=[HBox(children=[self.buttonStore,self.buttonDelete,self.buttonDoit]), self.textKey, self.textareaValue])
    self.vboxMain       = VBox        (children=[self.boxButtons, self.vboxControls, self.tabElements], layout=Layout(width='auto'))
    self.textKey        .observe      (self.keyChange, names='value')
    self.tabElements    .observe      (self.tabSelect, names='selected_index')
    self.buttonStore    .on_click     (self.storeItem)
    self.buttonDelete   .on_click     (self.deleteItem)
    self.buttonDoit     .on_click     (self.doitClick)
    self.buildTab ()

sample  = {'a':'one','b':'two'}
doit    = lambda w, d: print (f'doit {type(w)} {d}')
d       = dictor(sample,doit)
d       .show()
print   (d.data)


# _sp: button box

In [ ]:
#@markdown <font size='+2' color='#005F6A'>**button box**</font><br>
#@markdown * An alternative for the Tab widget for selecting **large amount of items**
#@markdown * Uses **Buttons** and a **Box** with **flex wrap layout**.
#@markdown * Provide a **mechanism for select** an item.
#@markdown ```python
#@markdown bbc  = lambda bb: print(bb.selecetd)    # event function: gets self as parameter
#@markdown bbox = ButtonBox (lst,                  # lst contains realy long descriptions
#@markdown                     color   = 'lavender', # selection color (default is powderblue)
#@markdown                     maxchar = 40,         # cut description position
#@markdown                     clicker = bbc)        # event
#@markdown display (bbox.box)                      # show widget
#@markdown ```

from ipywidgets import Button, Box, Layout

class ButtonBox ():
  """ An alternative for the Tab widget
  """
  def _clicker (self, b):

    # search clicked button and remember
    for i, description in enumerate (self.descriptions):
      if b.tooltip == description:
        self.selected = i
        self.button   = b

    # unselect (color) all buttons and select new (list of colors here: https://www.quackit.com/css/css_color_codes.cfm)
    for b in self.buttons: b.style={'button_color':None}
    self.buttons[self.selected].style={'button_color':self.color}

    # fire event
    if self.clicker: self.clicker (self)

  def __init__ (self, descriptions, clicker=None, maxchar=80, color='powderblue'):

    # remember stuff - list to set
    self.descriptions = descriptions if len (descriptions) == len (set (descriptions)) else set (descriptions)
    self.clicker, self.maxchar, self.color, self.selecetd = clicker, maxchar, color, None

    # make buttons
    self.buttons = [Button (description = i if len (i) <= maxchar else f'{i[:(maxchar-3)]}...',
                            layout      = Layout(width='auto', height='21px'),
                            tooltip     = f'{i}')
                    for i in descriptions]

    # put them in a box
    self.box = Box (layout   = Layout (display='flex', flex_flow='wrap'),
                    children = self.buttons)

    # bind event
    for button in self.buttons: button.on_click (self._clicker)

# *******************************************************************************************************************************************************************************************
# ****usage**********************************************************************************************************************************************************************************
# *******************************************************************************************************************************************************************************************
lst = ['1lskdjf lskdjf slkxdfj lsdkfj d slkdfj lskdfj kdkkd lsdkfj dkk', '2lskdjf skdfj kdkkd lsdkfj dkk', '3lskdjf slkcdfj lskdfj kdkkd lsdkfj dkk', '4lskdjf slkdfj  kdkykd lsdkfj dkk',
       '5Three dots at end lskdjf lskdjf slkdfj  lsdkfj d slkdfj lskdfj kdkkd lsdkfj  lsdkfj d slkdfj lskdfj kdkkd lsdkfj lskdfj kdkkd lsdkfj d slkdfj lskdfj kdkkd lsdkfj dkk',
       '6lskdjf slkdfj lskdfj kddkkd lsdkfj dkk', '7lskdjf sd sdfyslkdfj lskdfj kdkkd lsdkfj dkk', '8lskdjf slkdfj lskadfj kdkkd lsdkfj dkk', '9lskdjf slkdfj  kdkkd ls4dkfj dkk',
       '10lskdjf slkdfj ls3kdfj kdkkd lsdkfj dkk', '11lskdjf sd sdfsslkdfj lskdfj kdkkd lsdkfj dkk', '12lskdjf slnkdfj lskdfj kdkkd lsdkfj dkk', '13lskdjf slkdfj  kd6kkd lsdkfj dkk',
       '14lskdjf slkdfj lskdfj kdk3kd lsdkfj dkk', '15lskdjf lskdjf slk8dfj lskdfj kdkkd lsdkfj d slkdfj lskdfj kdkkd lsdkfj dkk', '16lsk9djf slkdfj   lsdkfj dkk', '17lskdjf']

bbc  = lambda bb: print(bb.selecetd)      # event function: get self as parameter
bbox = ButtonBox (lst,                    # lst contains realy long descriptions
                  color   = 'lavender',   # selection color (default is powderblue)
                  maxchar = 40,           # cut description position
                  clicker = bbc)          # event
display (bbox.box)                        # show widget

#_sp: breakpoint

In [ ]:
#@markdown <font size='+2' color='#005F6A'>**debug**</font><br>
#@markdown * When an error happens you can run **%debug** in a new code cell.
#@markdown * Or you set a breakpoint with ```from IPython.core.debugger import Pdb; Pdb().set_trace()```
#@markdown * Some commands:<br><table align='left'><tr><td><b>'q': quit</b> - Exit the debugger.<br>_</td><td><b>'c': continue</b> - Run to the next breakpoint.<br>_</td><td><b>'l' : list</b> - Shows 11 lines of code<br>around the actual position.</td></tr><tr><td><b>'n': next</b> - Run the next line<br>in current function.</td><td><b>'s': step</b> - step in to a function.<br>_</td><td><b>'r' : return</b> - Continue execution until<br>the current function returns.</td></tr></tr><tr><td><b>'h': help</b> - Shows help for a command<br>or all (if none parameter)</td><td><b>'b': breakpoint</b> - Makes a breakpoint for<br>a function or codeline (nr).</td><td><b>'cl' : clear</b> - Deletes breakpoint(s)<br>_</td></tr></table>

# breakpoint
# from IPython.core.debugger import Pdb; Pdb().set_trace()
pass

#_sp: breakpoint - short

In [ ]:
#from IPython.core.debugger import Pdb; Pdb().set_trace()

#_sp: escape codes

In [ ]:
#@markdown <font size='+2' color='#005F6A'>**escape codes**</font><br>
#@markdown * selection [from here](https://gist.github.com/iansan5653/c4a0b9f5c30d74258c5f132084b78db9)
#@markdown * [wikipedia](https://en.wikipedia.org/wiki/ANSI_escape_code) / [python docs](https://docs.python.org/3/reference/lexical_analysis.html#escape-sequences)
#@markdown ```python
#@markdown #        +---------------------------* '\x': hex code as character ('\u': unicode16, '\U': unicode32 )
#@markdown #        | +-------------------------* '1b': code for ESC sequence (decimal 27)
#@markdown #        | |+------------------------* '[': start parameter sequence
#@markdown #        | ||       +----------------* ';': seperated parameter
#@markdown #        | ||       |+---------------* 'm': end of parameter sequence
#@markdown #        | ||       ||  +------------* content
#@markdown #        | ||       ||  |     +------* '0': set sequence back
#@markdown #        | ||       ||  |     |
#@markdown print ("\x1b[1;3;4;34mabc\x1b[0mxyz")
#@markdown ```
#@markdown <font color='blue'><b><i><u>abc</u></i></b></font>xyz
print("\x1b[1;3;4;34mabc\x1b[0mxyz")
print('\x1b[0m Reset / Normal \x1b[0m')
print('\x1b[1m Bold or increased intensity \x1b[0m')
print('\x1b[3m Italic \x1b[0m')
print('\x1b[4m Underline \x1b[0m')

print(' Basic Foreground Colors:')
print('\t\x1b[30m Black foreground\x1b[0m')
print('\t\x1b[31m Red foreground\x1b[0m')
print('\t\x1b[32m Green foreground\x1b[0m')
print('\t\x1b[33m Yellow foreground\x1b[0m')
print('\t\x1b[34m Blue foreground\x1b[0m')
print('\t\x1b[35m Magenta foreground\x1b[0m')
print('\t\x1b[36m Cyan foreground\x1b[0m')
print('\t\x1b[37m White foreground\x1b[0m')
print('\t\x1b[39m Default foreground color \x1b[0m')

print(' Basic Background Colors:')
print('\t\x1b[40m Black background\x1b[0m')
print('\t\x1b[41m Red background\x1b[0m')
print('\t\x1b[42m Green background\x1b[0m')
print('\t\x1b[43m Yellow background\x1b[0m')
print('\t\x1b[44m Blue background\x1b[0m')
print('\t\x1b[45m Magenta background\x1b[0m')
print('\t\x1b[46m Cyan background\x1b[0m')
print('\t\x1b[47m White background\x1b[0m')
print('\t\x1b[49m Default background color \x1b[0m')

print(' Bright Foreground Colors:')
print('\t\x1b[90m Bright Black foreground\x1b[0m')
print('\t\x1b[91m Bright Red foreground\x1b[0m')
print('\t\x1b[92m Bright Green foreground\x1b[0m')
print('\t\x1b[93m Bright Yellow foreground\x1b[0m')
print('\t\x1b[94m Bright Blue foreground\x1b[0m')
print('\t\x1b[95m Bright Magenta foreground\x1b[0m')
print('\t\x1b[96m Bright Cyan foreground\x1b[0m')
print('\t\x1b[97m Bright White foreground\x1b[0m')

print(' Bright Background Colors:')
print('\t\x1b[100m Bright Black background\x1b[0m')
print('\t\x1b[101m Bright Red background\x1b[0m')
print('\t\x1b[102m Bright Green background\x1b[0m')
print('\t\x1b[103m Bright Yellow background\x1b[0m')
print('\t\x1b[104m Bright Blue background\x1b[0m')
print('\t\x1b[105m Bright Magenta background\x1b[0m')
print('\t\x1b[106m Bright Cyan background\x1b[0m')
print('\t\x1b[107m Bright White background\x1b[0m')

#_sp: youtube

In [ ]:
#@markdown <font size='+2' color='#005F6A'>**youtube**</font><br>
from IPython.display import YouTubeVideo

jukebox = {"The Dead South - In Hell I'll Be In Good Company"   :"B9FzVhw8_bY",
           "Big Bad Voodoo Daddy - Why Me?"                     :"a3Z4RWZa9WA",
           "Kishi Bashi - Violin Tsunami"                       :"xlXwpaAVoJQ",
           "Jamie T - Sheila"                                   :"4-L7Cadb-c0",
           "Tash Sultana - Jungle"                              :"1ExkpBpYEPw"}

select = "Tash Sultana - Jungle" #@param ["The Dead South - In Hell I'll Be In Good Company","Big Bad Voodoo Daddy - Why Me?","Kishi Bashi - Violin Tsunami","Jamie T - Sheila","Tash Sultana - Jungle"]

display ( YouTubeVideo ( jukebox [ select ] ) )

#_sp: radio buttons

In [ ]:
#@markdown <font size='+2' color='#005F6A'>**radio buttons**</font><br>
import ipywidgets as widgets
import numpy


output_radio_selected = widgets.Text()
radio1 = widgets.RadioButtons(options=['mse'])
radio2 = widgets.RadioButtons(options=['mae'])

radio1.index = None
radio2.index = None

def radio1_observer(sender):
    #print(sender)
    radio2.unobserve(radio2_observer, names=['value'])
    radio2.index = None

    global selected_option
    output_radio_selected.value = radio1.value
    selected_option = output_radio_selected.value
    print('Selected option set to: ' + selected_option)

    radio2.observe(radio2_observer, names=['value'])

def radio2_observer(sender):
    radio1.unobserve(radio1_observer, names=['value'])
    radio1.index = None

    global selected_option2
    output_radio_selected.value = radio2.value
    selected_option2 = output_radio_selected.value
    print('Selected option set to: ' + selected_option2)

    radio1.observe(radio1_observer, names=['value'])


radio1.observe(radio1_observer, names=['value'])
radio2.observe(radio2_observer, names=['value'])

widgets.HBox([radio1,radio2])

#_sp: svg with graphviz

In [ ]:
#@markdown <font size='+2' color='#005F6A'>**svg with graphviz**</font><br>
import graphviz
s='init->predict->loss->gradient->step->stop\nstep->predict[label=repeat]'
graphviz.Source('digraph G{ rankdir="LR"' + s + '; }')

#_sp: gdrive links


In [ ]:
#@markdown <font size='+2' color='#005F6A'>**gdrive links / hosting**</font><br>
#@markdown In gdrive get via context menu the share link. It looks like:
#@markdown > ```https://drive.google.com/file/d/1cehK_***************************/view?usp=sharing```

#@markdown Change it to the following format:
#@markdown > ```https://drive.google.com/uc?id=1cehK_***************************```

#@markdown you can use it in an image tag: ```<img src='your link'>```

print ('* link prefix: https://drive.google.com/uc?id=')
print ("* html: <img src='https://drive.google.com/uc?id=XXX' width='400' alt='alt text'>")
print ('* markdown: ![alt text](your link)')






# _sp: tex and greek letters

In [ ]:
#@markdown <font size='+2' color='#005F6A'>**tex and greek letters**</font><br>
#@markdown You get this: &emsp;&emsp; $\large \sum {\frac {Frit^z} {\sqrt[the] {C\alpha t}} }$<br> with that:&emsp;&emsp;&emsp;&ensp; ```$\large \sum {\frac {Frit^z} {\sqrt[the] {C\alpha t}} }$```
#
#@markdown ----
#@markdown <table><tr><td>Alpha $\alpha$ / $A$</td><td>Beta $\beta$ / $B$</td><td>Gamma $\gamma$ / $\Gamma$</td><td>Delta $\delta$ / $\Delta$</td><td>Epsilon $\epsilon$ / $E$</td><td>Zeta $\zeta$ / $Z$</td></tr>
#@markdown <tr><td>Eta $\eta$ / $E$</td><td>Theta $\theta$ / $\Theta$</td><td>Iota $\iota$ / $I$</td><td>Kappa $\kappa$ / $K$</td><td>Lambda $\lambda$ / $\Lambda$</td><td>My $\mu$ / $M$</td></tr>
#@markdown <tr><td>Ny $\nu$ / $N$</td><td>Xi $\xi$ / $\Xi$</td><td>Omikron $\omicron$ / $O$</td><td>Pi $\pi$ / $\Pi$</td><td>Rho $\rho$ / $R$</td><td>Sigma $\sigma$ / $\Sigma$</td></tr>
#@markdown <tr><td>Tau $\tau$ / $T$</td><td>Ypsilon $\upsilon$ / $\Upsilon$</td><td>Phi $\phi$ / $\Phi$</td><td>Chi $\chi$ / $X$</td><td>Psi $\psi$ / $\Psi$</td><td>Omega $\omega$ / $\Omega$</td></tr></table>

for i in range (0x03b1, 0x03c7): print (chr(i), end=' ')
print()
for i in range (0x0391, 0x03a7): print (chr(i), end=' ')


#_sp: text cell snippets

In [ ]:
#@markdown <font size='+2' color='#005F6A'>**text cell snippets - a little hack**</font><br>
#@markdown * run the cell and copy the output in a colab text cell and you'll get something like this:
#@markdown <table><tr><td>r1d1</td><td>r1d2</td><td>r1d3</td><td>r1d4</td></tr><tr><td>r2d1</td><td colspan=2 rowspan=2 align=center>
#@markdown $$\rule{7cm}{0.4pt}$$ $\left.{\;\\\;\\}\right\vert$ r2d2 - colspan2 - rowspan2$\left.{\;\\\;\\}\right\vert$<br>$$\rule{7cm}{0.4pt}$$
#@markdown </td><td>r2d3</td></tr><tr><td>r3d1</td><td>r3d2</td></tr><tr><td>r4d1</td><td>r4d2</td><td>r4d3</td><td>r4d4</td></tr></table>
from ipywidgets import Text, Textarea, Layout
Textarea (description='copy', layout=Layout(width='auto',height='800px'), value="""
**tables and lines**

---
<table>
<tr><td>r1d1</td><td>r1d2</td><td>r1d3</td></tr>
<tr><td>r2d1</td><td>r2d2</td><td>r2d3</td></tr>
<tr><td>r3d1</td><td>r3d2</td><td>r3d3</td></tr>
</table>

---
<table>
<tr><td colspan=3>r1d1 colspan3</td></tr>
<tr><td>r2d1</td><td>r2d2</td><td>r2d3</td></tr>
<tr><td>r3d1</td><td>r3d2</td><td>r3d3</td></tr>
</table>

---
<table>
<tr><td>r1d1</td><td rowspan=3>r1d2<br>rowspan3</td><td>r1d3</td></tr>
<tr><td>r2d1</td><td>r2d2</td></tr>
<tr><td>r3d1</td><td>r3d2</td></tr>
</table>

---
<table>
<tr><td>r1d1</td><td>r1d2</td><td>r1d3</td><td>r1d4</td></tr>
<tr><td>r2d1</td><td colspan=2 rowspan=2>r2d2<br>colspan2<br>rowspan2</td><td>r2d3</td></tr>
<tr><td>r3d1</td><td>r3d2</td></tr>
<tr><td>r4d1</td><td>r4d2</td><td>r4d3</td><td>r4d4</td></tr>
</table>

---
I - inner table / O - outer table
<table>
<tr><td>O r1d1</td><td>O r1d2</td><td>O r1d3</td><td>O r1d4</td></tr>
<tr><td>O r2d1</td><td colspan=2 rowspan=2 align=center>O r2d2  colspan2 rowspan2
<table>
<tr><td>I r1d1</td><td>I r1d2</td><td>I r1d3</td><td>I r1d4</td></tr>
<tr><td>I r2d1</td><td colspan=2 rowspan=2 align=center>
I r2d2<br>colspan2<br>rowspan2
</td><td>I r2d3</td></tr>
<tr><td>I r3d1</td><td>I r3d2</td></tr>
<tr><td>I r4d1</td><td>I r4d2</td><td>I r4d3</td><td>I r4d4</td></tr>
</table>
</td><td>O r2d3</td></tr>
<tr><td>O r3d1</td><td>O r3d2</td></tr>
<tr><td>O r4d1</td><td>O r4d2</td><td>O r4d3</td><td>O r4d4</td></tr>
</table>

---
<table>
<tr><td>r1d1</td><td>r1d2</td><td>r1d3</td><td>r1d4</td></tr>
<tr><td>r2d1</td><td colspan=2 rowspan=2 align=center>
$$\\rule{7cm}{0.4pt}$$
$\\left.{\\\\;\\\\\\\\\\\\;\\\\\\\\\\;\\\\\\;\\\\}\\right\\vert$
r2d2 - colspan2 - rowspan2$\\left.{\\;\\\\\\;\\\\\\;\\\\\\;\\\\}\\right\\vert$<br>
$$\\rule{7cm}{0.4pt}$$
</td><td>r2d3</td></tr>
<tr><td>r3d1</td><td>r3d2</td></tr>
<tr><td>r4d1</td><td>r4d2</td><td>r4d3</td><td>r4d4</td></tr>
</table>

---
<table><tr><td rowspan=4>
$\\left.{\\;\\\\\\;\\\\\\;\\\\}\\right\\vert$

</td><td colspan=2>$\\rule{8cm}{0.4pt}$</td><td rowspan=4>
$\\left.{\\;\\\\\\;\\\\\\;\\\\}\\right\\vert$

</td></tr >
<tr><td  align=center>r2d1</td><td  align=center>r2d2</td></tr>
<tr><td  align=center>r3d1</td><td  align=center>r3d2</td></tr>
<tr><td colspan=2>
$\\rule{8cm}{0.4pt}$

</td></tr></table>

---
<table><tr><td>

$\\left.{\\;}\\right\\vert$</td><td>

$\\left.{\\;\\\\}\\right\\vert$</td><td>

$\\left.{\\;\\\\\\;\\\\}\\right\\vert$</td><td>

$\\left.{\\;\\\\\\;\\\\\\;\\\\}\\right\\vert$</td><td>

$\\left.{\\;\\\\\\;\\\\\\;\\\\\\;\\\\\\;\\\\}\\right\\vert$</td><td>

$\\left.{\\;\\\\\\;\\\\\\;\\\\\\;\\\\\\;\\\\\\;\\\\}\\right\\vert$</td><td>

$\\left.{\\;\\\\\\;\\\\\\;\\\\\\;\\\\\\;\\\\\\;\\\\\\;\\\\}\\right\\vert$</td><td>

$\\left.{\\;\\\\\\;\\\\\\;\\\\\\;\\\\\\;\\\\\\;\\\\\\;\\\\\\;\\\\}\\right\\vert$</td><td align=center>

$\\rule{8cm}{0.4pt}$<br>
$\\rule{10cm}{1pt}$<br>
$\\rule{12cm}{4pt}$<br>
$\\rule{10cm}{1pt}$<br>
$\\rule{8cm}{0.4pt}$<br>

</td></tr></table>

""")


# _sp: vectorize

In [ ]:
#@markdown <font size='+2' color='#005F6A'>**vectorize**</font><br>
#@markdown <img src='https://drive.google.com/uc?id=1cHGRQjBYpeXUzEGo53yYfg7mSYjJDjTh' width='400' alt='vectorize.png'><br>
#@markdown ```python
#@markdown np.vectorize (lambda w: loss(nn(x,w), t) ) (ws)
#@markdown ```
import numpy as np

np.random.seed(seed=42)

def f(x)          : return x * 2
def nn(x, w)      : return x * w
def loss(y, t)    : return np.mean((t - y)**2)

x                 = np.random.uniform (0, 1, 20)          # 20 values between 0 and 1 uniform distributed
noise             = np.random.randn (x.shape[0]) * 0.2    # 20 standard normal distribution
t                 = f(x) + noise                          # noisy result (y)
ws                = np.linspace(0, 4, num=100)            # weight vector to vectorize

result = np.vectorize (lambda w: loss(nn(x,w), t) ) (ws)  # for each element in vector ws calculate via nn the y - and with that and t the loss
                                                          # we calculate a loss vector for the passed vector (ws) related to an x and t
result.shape, result

